In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import sys
sys.path.insert(0, '../')

In [ ]:
import numpy as np
import pandas as pd
from dotenv import load_dotenv

load_dotenv('../upload.env')

EMAIL = os.getenv('EMAIL')  # the e-mail you used to sign up
BUCKET_NAME = os.getenv('BUCKET_NAME') # you received it in your e-mail
PARTICIPANT_ID = os.getenv('PARTICIPANT_ID') # you received it in your e-mail
AWS_ACCESS_KEY = os.getenv('AWS_ACCESS_KEY') # you received it in your e-mail
AWS_SECRET_KEY = os.getenv('AWS_SECRET_KEY') # you received it in your e-mail

In [ ]:
from submission.EvalRSRunner import EvalRSRunner

In [ ]:
class RandomModel:
    def __init__(self, items: pd.DataFrame):
        self.items = items

    def predict(self, user_ids: pd.DataFrame, k=20) -> pd.DataFrame:
        user_ids = user_ids['user_id'].values
        num_users = len(user_ids)
        pred = self.items.sample(n=k*num_users, replace=True)['track_id'].values
        pred = pred.reshape(num_users, k )
        pred = np.concatenate((np.array(user_ids).reshape(-1,1), pred), axis=1)
        return pd.DataFrame(pred, columns=['user_id', *[ str(i) for i in range(k)]]).set_index('user_id')

In [ ]:
class RandomEvalRSRunner(EvalRSRunner):
    def train_model(self, train_df: pd.DataFrame):
        return RandomModel(self.df_tracks)

In [ ]:
runner = RandomEvalRSRunner(path_to_dataset='../lfm_1b_dataset',
                            num_folds=4,
                            aws_access_key_id=AWS_ACCESS_KEY,
                            aws_secret_access_key=AWS_SECRET_KEY,
                            participant_id=PARTICIPANT_ID,
                            bucket_name=BUCKET_NAME,
                            email=EMAIL)

In [ ]:
runner.evaluate(upload=True, limit=10000000)